In [1]:
import yaml
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics, model_selection, ensemble


pd.set_option('display.max_columns', None)

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
try:
    from CustomPipeline import *
except:
    import sys
    sys.path.insert(0,'/content/sample_data')
    from CustomPipeline import *
    print("ok")

In [3]:
try:
    train = pd.read_csv("./Datasets/train_anomaly.csv", delimiter=",", sep='.', index_col='id')
    test = pd.read_csv("./Datasets/test_anomaly.csv", delimiter=",", sep='.', index_col='id')
except:
    train = pd.read_csv("./sample_data/train_anomaly.csv", delimiter=",", sep='.')

In [4]:
with open("./Configs/params_model_linear.yml", 'r') as stream:
    try:
        params_linear = yaml.safe_load(stream)
        print(params_linear)
    except yaml.YAMLError as exc:
        print(exc)

{'alpha': 45.97949266795876}


In [5]:
RANDOM_STATE = 42

In [6]:
train["target"] = 100 * train["target"]

In [7]:
train, test_ensemble = train_test_split(train, test_size=0.1, random_state=RANDOM_STATE)
train_estimators, train_ensemble = train_test_split(train, test_size=0.2, random_state=RANDOM_STATE)

In [8]:
X = train_estimators.drop(["target"], axis=1)
y = train_estimators["target"]

# linear ridge

In [11]:
n_bins=63

X_lr = train_estimators.query("target > 600").drop(["target"], axis=1)
y_lr = train_estimators.query("target > 600")["target"]

lr = linear_model.Ridge(**params_linear)
model = LinearWrapper(lr, bins_linear=n_bins)
model.fit(X_lr, y_lr)

# save model

In [12]:
import pickle
import os

directory = './Models'
if not os.path.exists(directory):
    os.makedirs(directory)

# save
with open('./Models/model_ridge.pkl','wb') as f:
    pickle.dump(model,f)

# load
with open('./Models/model_ridge.pkl', 'rb') as f:
    model = pickle.load(f)


In [13]:
pred = model.predict(train_ensemble.drop(['target'], axis=1))
train_ensemble['lr'] = pred

pred = model.predict(test_ensemble.drop(['target'], axis=1))
test_ensemble['lr'] = pred

pred = model.predict(test)
test['lr'] = pred

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
